In [160]:
#packages
import numpy as np
import numpy.ma as ma
import sys
import os
import shapely
import scipy 

from scipy import signal
from skimage.draw import polygon2mask
from shapely import Polygon
from shapely import affinity
from matplotlib import pyplot as plt
from shapely.geometry.polygon import Polygon
from PIL import Image
from matplotlib.backends.backend_pdf import PdfPages

Load data:

In [161]:
suite2p_folder = 'C:/Users/Tóth Mátyás/Desktop/szakdolgozat/example_dendriric recording_numpy'
cell = 'c1'
numbers = [0,1,2,3,4,5,6]

In [162]:
imgs = np.load('C:/Users/Tóth Mátyás/Desktop/szakdolgozat/example_dendriric recording_numpy/plane0/4.npy')
polygons = np.load('C:/Users/Tóth Mátyás/Desktop/szakdolgozat/example_dendriric recording_numpy/plane0/c1/polygons.npy', allow_pickle = True)
roi_masks = np.load('C:/Users/Tóth Mátyás/Desktop/szakdolgozat/example_dendriric recording_numpy/c1/masks.npy', allow_pickle = True)

masks = np.load(suite2p_folder + '/plane0/mean_img.npy', allow_pickle=True)

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

Also, load preprocessed data from an earliear phase of the research:

In [163]:
sm0 = np.load('Asmooth1.npy', allow_pickle = True)
sm1 = np.load('Asmooth2.npy', allow_pickle = True)
sm2 = np.load('Asmooth3.npy', allow_pickle = True)
sm3 = np.load('Asmooth4.npy', allow_pickle = True)
sm4 = np.load('Asmooth5.npy', allow_pickle = True)
sm5 = np.load('Asmooth6.npy', allow_pickle = True)
sm6 = np.load('Asmooth7.npy', allow_pickle = True)

ev0 = np.load('Aevent1.npy', allow_pickle = True)
ev1 = np.load('Aevent2.npy', allow_pickle = True)
ev2 = np.load('Aevent3.npy', allow_pickle = True)
ev3 = np.load('Aevent4.npy', allow_pickle = True)
ev4 = np.load('Aevent5.npy', allow_pickle = True)
ev5 = np.load('Aevent6.npy', allow_pickle = True)
ev6 = np.load('Aevent7.npy', allow_pickle = True)

Pad the signals:

In [164]:
pad_signal = np.zeros(50)
sm0 = np.append(sm0, pad_signal)
sm1 = np.append(sm1, pad_signal)
sm2 = np.append(sm2, pad_signal)
sm3 = np.append(sm3, pad_signal)
sm4 = np.append(sm4, pad_signal)
sm5 = np.append(sm5, pad_signal)
sm6 = np.append(sm6, pad_signal)

In [165]:
pad_len = 25
zero_pad_array = 512*512*pad_len
pad_dF = np.zeros(zero_pad_array)
pad_dF = pad_dF.reshape(pad_len,512,512)
imgs = np.append(imgs, pad_dF, axis=0)
movie_data = imgs.copy()

In [166]:
# def bufferize(mask, distance = 25):
#     mask = Polygon(mask)
#     mask_exp = mask.buffer(distance)
#     mask_exp_list = list(mask_exp.exterior.coords)
#     mask_nparray = np.array(np.round(mask_exp_list))
#     return mask_nparray.astype(int)

Create bigger ROIs and implement the event detection function:

In [167]:
def roi_mask_expander(roi, d = 2):
    
    roi = list(roi)
    np.unique(np.array(roi), axis = 0)
    
    for i in np.arange(len(roi)):
        x = roi[i][0]
        y = roi[i][1]
        roi.append([x+d,y])
        roi.append([x-d,y])
        roi.append([x,y+d])
        roi.append([x,y-d])
          
    new_roi = np.unique(np.array(roi), axis=0)
    
    return new_roi

In [168]:
prof0 = roi_mask_expander(roi_masks[0],6) #soma
prof1 = roi_mask_expander(roi_masks[1],5)
prof2 = roi_mask_expander(roi_masks[2],5)
prof3 = roi_mask_expander(roi_masks[3],5)
prof4 = roi_mask_expander(roi_masks[4],5)
prof5 = roi_mask_expander(roi_masks[5],5)
prof6 = roi_mask_expander(roi_masks[6],5)

Event detection function:

In [169]:
def event_detector(signal, theta_upper = 1.65, theta_lower = 1.6): 
    
    N = len(signal)
    
    event = np.zeros(N)
    transient = np.zeros(N)
    start_end = []
    
    refr = 0
    start_end_i = np.zeros(2)
    
    for i in np.arange(1,N):
        if ((signal[i] > theta_upper) and (signal[i-1] < theta_upper) and (refr == 0)):
            event[i] = 1 
            refr = 1
            start_end_i[0] = i
        if ((signal[i]<theta_lower) and (signal[i-1]>theta_lower) and (refr == 1)):
            refr = 0
            start_end_i[1] = i
            start_end.append(start_end_i)
            start_end_i = np.zeros(2)
        if (refr == 1):
            transient[i] = 1
    if ((start_end_i[0] > 0) & (start_end_i[1] == 0)):
        start_end_i[1] = N
        start_end.append(start_end_i)
    
    return event, transient, start_end

## Here we specifiy the threshold of the detected transient event:

In [170]:
# sig1_event = event_detector(signal1_smooth, 0.08, 0.007) # 1st. 0.08
# sig2_event = event_detector(signal2_smooth, 0.12, 0.01) #1st 0.15
# sig3_event = event_detector(signal3_smooth, 0.2, 0.007) #1st 0.25
# sig4_event = event_detector(signal4_smooth, 0.1, 0.05) #1st 0.15
# sig5_event = event_detector(signal5_smooth, 0.12, 0.001)
# sig6_event = event_detector(signal6_smooth, 0.2, 0.005)
# sig7_event = event_detector(signal7_smooth, 0.1, 0.005)

In [171]:
#detect the events
soma_signal = sm0
actual_signal = sm3

threshold_upper = 0.1
threshold_lower = 0.05

# soma_event, soma_transient, soma_start_end = event_detector(soma_signal, threshold_upper, threshold_lower)
event, transient, start_end = event_detector(actual_signal, threshold_upper, threshold_lower)

In [172]:
#uncomment the relevant compartment!

# soma_lista_lower = np.array(soma_start_end)[:,0].astype(int)
# soma_lista_upper = np.array(soma_start_end)[:,1].astype(int)

lista_lower = np.array(start_end)[:,0].astype(int)
lista_upper = np.array(start_end)[:,1].astype(int)

In [173]:
#save the begining of transient event indexes if needed!

# soma_list_lower_save = np.save('soma_list_lower', soma_list_lower)

# dend_list_lower_save1 = np.save('dend_list_lower1', list_lower)
# dend_list_lower_save2 = np.save('dend_list_lower2', list_lower)
# dend_list_lower_save3 = np.save('dend_list_lower3', list_lower)
# dend_list_lower_save4 = np.save('dend_list_lower4', list_lower)
# dend_list_lower_save5 = np.save('dend_list_lower5', list_lower)
# dend_list_lower_save6 = np.save('dend_list_lower6', list_lower)

Functions for generating transient profiles and for create a pdf file for all the transient profiles of the selected neural compartment:

In [174]:
def transient_profile(transient_signal, movie_tensor, base_pic, start = 1, finish = 1):
    
    if finish >= len(transient_signal)-1:
        finish = len(transient_signal)-6
    
    index = np.arange(start-5, finish+5)
    index_len = np.arange(0,len(index),1)
    transient = transient_signal[index]
    movie_tensor = movie_tensor[index,:,:]
    W = transient[:, np.newaxis, np.newaxis] * movie_tensor #weighted_frames
    transient_pic = np.sum(W[index_len,:,:],axis=0)
    normalized_trans_pic = transient_pic/sum(transient**2)
    trans = normalized_trans_pic
    return trans

Some preprocess:

In [175]:
# if lista_lower[0] < 6:
#     lista_lower[0] = 6

Xlim and Ylim source profile window coordinates for plotting a unique transient profile each compartment:

In [176]:
xmin_soma, xmax_soma = 300, 400 # it is inverted, so we need to give inverted coordinates
ymin_soma, ymax_soma = 500, 400

In [177]:

xminROI1, xmaxROI1 = 390, 460
yminROI1, ymaxROI1 = 320, 230

xminROI2, xmaxROI2 = 335, 365 
yminROI2, ymaxROI2 = 300, 250

xminROI3, xmaxROI3 = 430, 500
yminROI3, ymaxROI3 = 250, 160

xminROI4, xmaxROI4 = 350, 400
yminROI4, ymaxROI4 = 260, 200

xminROI5, xmaxROI5 = 330, 355
yminROI5, ymaxROI5 = 280, 220

xminROI6, xmaxROI6 = 330, 370 
yminROI6, ymaxROI6 = 260, 130

Set the time - which sequence are we in.

In [178]:
# which sequence - 1

which_sequence = 4

Transient profile creation function for the soma:

In [179]:
def show_profiles_soma(prof_mask, signal_smooth, event, sp, bp, index_start,
    index_stop, xone, xtwo, yone, ytwo, want, offset = 50, 
    og_poly_number = 0, w_seq = 4, title = '{0}. somatic transient', soma_title = '', name=''):
    
    l = len(index_start)
    for i in range(l):
        
        trans = transient_profile(signal_smooth, sp, bp, index_start[i], index_stop[i])
        
        # polygons =  np.load(suite2p_folder + '/plane0/' + str(cell) + '/polygons.npy', allow_pickle=True)
        # poly_points = polygons[og_poly_number]
        # poly_points = polygon
        # polygon_shape = Polygon(polygon)
        # original_polygon = polygon_shape.exterior.xy
      
        buffered_mask = np.array(prof_mask)
        mask = polygon2mask(trans.shape, buffered_mask)
        result = ma.masked_array(trans, np.invert(mask))
        
        base_result = ma.masked_array(bp, np.invert(mask))
        
        x_var = trans[tuple(prof_mask.T)].tolist()
        y_var = bp[tuple(prof_mask.T)].tolist()
        cor = np.round(np.corrcoef(x_var, y_var)[0][1], 3)
        
        
        interval_min = offset
        
        if index_start[0] == 6:
             interval_min[0] = 1
                
        if signal_smooth[index_start[i]+2] > 2:
            interval_min = offset +20
        else:
            interval_min = offset
            
        interval_max = interval_min + (index_stop[i]-index_start[i]) + 1
    
        #time scale parameters
        len_signal = len(signal_smooth)
        len_signal = len_signal*w_seq
        current = 112*w_seq
        time_len_s = round(current/len_signal,2)
        start_point = index_start[i]-offset
        start_point = start_point+len_signal
        end_point = index_stop[i]+offset
        end_point = end_point+len_signal
        time_interval = round((end_point-start_point)/5)
        lbls = [i*time_len_s for i in list(range(start_point, end_point, time_interval))] #start_point
        lbls = [round(num, 1) for num in lbls]
        tcks = list(range(0, end_point-start_point, time_interval))
        
        fig = plt.figure(i)
        fig.set_figheight(12)
        fig.set_figwidth(16)
        
        plt.subplots_adjust(wspace=0.1)
        plt.subplots_adjust(hspace=0.8)
        
        rows = 2
        columns = 3
        
        fig.add_subplot(rows, columns, 1)
        plt.imshow(bp)
        flipped_polygon = np.flip(prof_mask, axis=None)
        flipped_ud_polygon = np.flipud(flipped_polygon)
        polygon = Polygon(flipped_ud_polygon)
        poly_points = polygon.buffer(0.7, join_style=3)
        polygon_shape = Polygon(poly_points)
        plt.plot(*polygon_shape.exterior.xy, color=colors[og_poly_number])
        plt.title('A - Total view \n (mean image)', fontsize=20)
        plt.axis('off')
        
        fig.add_subplot(rows, columns, 2)
        plt.imshow(base_result)
        plt.xlim(xone, xtwo)
        plt.ylim(yone, ytwo)
        plt.title('B - Source profile \n (mean image)', fontsize=20)
        plt.axis('off')
        
        ax = fig.add_subplot(rows, columns, 3)
        ax.imshow(result)
        # plt.plot(*original_polygon, 'r')
        plt.xlim(xone, xtwo)
        plt.ylim(yone, ytwo)
        plt.title('C - Transient profile \n r = {0}'.format(cor), fontsize=20)
        plt.xticks(color = 'w')
        plt.yticks(color = 'w')
        
        want = want
        want_i = [index_start[k] for k in want]
        
        color = "white" if index_start[i] in want_i else "red"
        for x in ["top","left","right","bottom"]:
            ax.spines[x].set_color(color)
            ax.spines[x].set_linewidth(8)
        
        fig.add_subplot(rows, columns, 4)
        plt.plot(signal_smooth[index_start[i]-offset:index_stop[i]+offset], 'k')
        plt.title(soma_title, fontsize=20)
        plt.ylim([-0.5, 2])
        plt.xlabel('Time (s)', fontsize = 15)
        plt.ylabel('Amplitude', fontsize = 15)
        plt.xticks(ticks = tcks, labels = lbls, fontsize = 15)
        plt.yticks(fontsize = 15)
        plt.axvspan(interval_min, interval_max, color='moccasin', alpha=0.7)
        
        title_num = 1
        title_num = title_num + i
        
        fig.suptitle(title.format(title_num), fontsize=20, weight = 'bold')
        
        pdf_pages = len(index_stop)
        
        with PdfPages(name) as pdf:
            for fig in range(pdf_pages):
                plt.figure(fig)
                pdf.savefig(bbox_inches="tight")

Transient profile creation function for the dendrite:

In [180]:
def show_profiles_dendrites(prof_mask, soma_signal_smooth, dendrite_signal_smooth, soma_event, dendrite_event, sp, bp, index_start,
    index_stop, xone, xtwo, yone, ytwo, want, offset = 50, og_poly_number = 0, w_seq = 4,
    title = '{0}. dendritic transient', soma_title ='', dendrite_title = '', name= ''):
    
    l = len(index_start)
    for i in range(l):
        
        trans = transient_profile(dendrite_signal_smooth, sp, bp, index_start[i], index_stop[i])
        
        # polygons =  np.load(suite2p_folder + '/plane0/' + str(cell) + '/polygons.npy', allow_pickle=True)
        # poly_points = polygons[og_poly_number]
        # poly_points = polygon
        # polygon_shape = Polygon(poly_points)
        # original_polygon = polygon_shape.exterior.xy
        # prop_cycle = plt.rcParams['axes.prop_cycle']
        # colors = prop_cycle.by_key()['color']
      
        buffered_mask = np.array(prof_mask)
        mask = polygon2mask(trans.shape, buffered_mask)
        result = ma.masked_array(trans, np.invert(mask))
        
        base_result = ma.masked_array(bp, np.invert(mask))
        
        x_var = trans[tuple(prof_mask.T)].tolist()
        y_var = bp[tuple(prof_mask.T)].tolist()
        cor = np.round(np.corrcoef(x_var, y_var)[0][1], 3)
        
        interval_min = offset
        
        if index_start[0] == 6:
             interval_min = 1
        
        if dendrite_signal_smooth[index_start[i]+2] > 2:
            interval_min = offset + 20
        else:
            interval_min = offset
            
        interval_max = interval_min + (index_stop[i]-index_start[i]) + 1
        
        #time-scale parameters
        len_signal = len(dendrite_signal_smooth)
        len_signal = len_signal*w_seq
        current = 112*w_seq
        time_len_s = round(current/len_signal,2)
        start_point = index_start[i]-offset
        start_point = start_point+len_signal
        end_point = index_stop[i]+offset
        end_point = end_point+len_signal
        time_interval = round((end_point-start_point)/5)
        lbls = [i*time_len_s for i in list(range(start_point, end_point, time_interval))] #start_point
        lbls = [round(num, 1) for num in lbls]
        tcks = list(range(0, end_point-start_point, time_interval))
        
        fig = plt.figure(i)
        fig.set_figheight(15)
        fig.set_figwidth(20)
        
        plt.subplots_adjust(hspace=0.5)
        
        rows = 2
        columns = 3
        
        fig.add_subplot(rows, columns, 1)
        plt.imshow(bp)
        flipped_polygon = np.flip(prof_mask, axis=None)
        flipped_ud_polygon = np.flipud(flipped_polygon)
        polygon = Polygon(flipped_ud_polygon)
        poly_points = polygon.buffer(0.7, join_style=3)
        polygon_shape = Polygon(poly_points)
        plt.plot(*polygon_shape.exterior.xy, color=colors[og_poly_number])
        plt.title('A - Total view \n (mean image)', fontsize = 20)
        plt.axis('off')
        
        fig.add_subplot(rows, columns, 2)
        plt.imshow(base_result)
        plt.xlim(xone, xtwo)
        plt.ylim(yone, ytwo)
        plt.title('B - Source profile \n (mean image)', fontsize = 20)
        plt.axis('off')
        
        ax = fig.add_subplot(rows, columns, 3)
        ax.imshow(result)
        # plt.plot(*original_polygon, color = colors[og_poly_number])
        plt.xlim(xone, xtwo)
        plt.ylim(yone, ytwo)
        plt.title('C - Transient profile \n r = {0}'.format(cor), fontsize = 20)
        plt.xticks(color = 'w')
        plt.yticks(color = 'w')
        
        want = want
        want_i = [index_start[k] for k in want]
        
        color = "white" if index_start[i] in want_i else "red"
        for x in ["top","left","right","bottom"]:
            ax.spines[x].set_color(color)
            ax.spines[x].set_linewidth(8)
            
        fig.add_subplot(rows, columns, 4)
        plt.plot(soma_signal_smooth[index_start[i]-offset:index_stop[i]+offset], 'k')
        plt.title(soma_title, fontsize = 20)
        plt.ylim([-0.5, 3])
        plt.xlabel('Time (s)', fontsize = 15)
        plt.ylabel('Amplitude', fontsize = 15)
        plt.xticks(ticks = tcks*w_seq, labels = lbls*w_seq, fontsize = 15)
        plt.yticks(fontsize = 15)
        plt.axvspan(interval_min, interval_max, color='moccasin', alpha=0.7)
        
        fig.add_subplot(rows, columns, 5)
        plt.plot(dendrite_signal_smooth[index_start[i]-offset:index_stop[i]+offset], 'k')
        plt.title(dendrite_title, fontsize = 20)
        plt.ylim([-0.5, 3])
        plt.xlabel('Time (s)', fontsize = 15)
        plt.ylabel('Amplitude', fontsize = 15)
        plt.xticks(ticks = tcks*w_seq, labels = lbls*w_seq, fontsize = 15)
        plt.yticks(fontsize = 15)
        plt.axvspan(interval_min, interval_max, color='moccasin', alpha=0.7)
        
        title_num = 1
        title_num = title_num + i
        
        fig.suptitle(title.format(title_num), fontsize=20, weight = 'bold')
        
        pdf_pages = len(index_stop)
        
        with PdfPages(name) as pdf:
            for fig in range(pdf_pages):
                plt.figure(fig)
                pdf.savefig(bbox_inches="tight")    

Calculate correlations:

In [181]:
def cor_coeffs(prof_mask, smooth_signal, movie_tensor, base_profile, list_a, list_b):
    cor_list = []
    n = len(list_a)
    for i, j in zip(range(n), range(n)):
        trans = transient_profile(smooth_signal, movie_tensor, base_profile, list_a[i], list_b[j])
        
        buffered_mask = np.array(prof_mask)
        mask = polygon2mask(trans.shape, buffered_mask)
        result = ma.masked_array(trans, np.invert(mask))
        base_result = ma.masked_array(base_profile, np.invert(mask))
        
        x_var = trans[tuple(prof_mask.T)].tolist()
        y_var = base_profile[tuple(prof_mask.T)].tolist()
        cor = np.round(np.corrcoef(x_var, y_var)[0][1], 3)
        
        cor_list.append(cor)
        
    return cor_list

Separate true and false correlations:

In [182]:
def truefalse(cor_coeffs, want):
    
    want = want
    
    false_cor = [cor_coeffs[k] for k in want]
    true_cor = np.setxor1d(cor_coeffs, false_cor)
    
    return true_cor, false_cor

## From here we specifiy the correlation list according to the actual compartment and the plots of the transient profiles of the actual compartment:

In [183]:
#the prof and the sm parameters in the cor_list_dendrite need to be modified according the the currently measured ROI

# cor_list_soma = cor_coeffs(prof0, sm0, movie_data, masks, soma_lista_lower, soma_lista_upper)
cor_list_dendrite = cor_coeffs(prof3, sm3, movie_data, masks, lista_lower, lista_upper)

Create PDFs:

In [184]:
# exception_list_soma = []

In [185]:
# true_cor_soma, false_cor_soma = truefalse(cor_list_soma, exception_list_soma)

# name_soma = '7_seq_soma_truecor.npy'
# name2_soma = '7_seq_soma_falsecor.npy'

# np.save(name_soma, true_cor_soma)
# np.save(name2_soma, false_cor_soma)

In [186]:
# show_profiles_soma(prof0, sm0, ev0, movie_data, masks, soma_lista_lower,
#                    soma_lista_upper, xmin_soma, xmax_soma, ymin_soma, ymax_soma, exception_list_soma, pad_len, 0, which_sequence,
#                    '7. sequence, {0}. somatic transient \n', 'D - Soma dF/F', '7. sequence, somatic events.pdf')

In [187]:
exception_list_dendrite = []

In [188]:
# true_cor_dendrite, false_cor_dendrite = truefalse(cor_list_dendrite, exception_list_dendrite)

# name_dendrite = '5_seq_dend_3_truecor.npy'
# name2_dendrite = '5_seq_dend_3_falsecor.npy'

# np.save(name_dendrite, true_cor_dendrite)
# np.save(name2_dendrite, false_cor_dendrite)

In [189]:
# show_profiles_dendrites(prof3, sm0, sm3, ev0, ev3, movie_data, masks, lista_lower,
#                         lista_upper, xminROI3, xmaxROI3, yminROI3, ymaxROI3, exception_list_dendrite, pad_len, 3, which_sequence, 
#                         '5. sequence, \n {0}. dendritic transient 3. ROI', 'D - Soma dF/F \n', 
#                         'E - Dendritic dF/F \n','5. sequence, dendritic events - 3. ROI.pdf')